In [1]:
#
###### https://github.com/LucaCanali/Miscellaneous/blob/master/Pyspark_SQL_Magic_Jupyter/IPython_Pyspark_SQL_Magic.py
#
#
# IPython magic functions to use with Pyspark and Spark SQL
# The following code is intended as examples of shorcuts to simplify the use of SQL in pyspark
# The defined functions are:
#
# %sql <statement>          - return a Spark DataFrame for lazy evaluation of the SQL
# %sql_show <statement>     - run the SQL statement and show max_show_lines (50) lines
# %sql_display <statement>  - run the SQL statement and display the results using a HTML table 
#                           - this is implemented passing via Pandas and displays up to max_show_lines (50)
# %sql_explain <statement>  - display the execution plan of the SQL statement
#
# Use: %<magic> for line magic or %%<magic> for cell magic.
#
# Author: Luca.Canali@cern.ch
# September 2016
#

from IPython.core.magic import register_line_cell_magic

# Configuration parameters
max_show_lines = 50         # Limit on the number of lines to show with %sql_show and %sql_display
detailed_explain = True     # Set to False if you want to see only the physical plan when running explain


@register_line_cell_magic
def sql(line, cell=None):
    "Return a Spark DataFrame for lazy evaluation of the sql. Use: %sql or %%sql"
    val = cell if cell is not None else line 
    return sqlContext.sql(val)

@register_line_cell_magic
def sql_show(line, cell=None):
    "Execute sql and show the first max_show_lines lines. Use: %sql_show or %%sql_show"
    val = cell if cell is not None else line 
    return sqlContext.sql(val).show(max_show_lines) 

@register_line_cell_magic
def sql_display(line, cell=None):
    """Execute sql and convert results to Pandas DataFrame for pretty display or further processing.
    Use: %sql_display or %%sql_display"""
    val = cell if cell is not None else line 
    return sqlContext.sql(val).limit(max_show_lines).toPandas() 

@register_line_cell_magic
def sql_explain(line, cell=None):
    "Display the execution plan of the sql. Use: %sql_explain or %%sql_explain"
    val = cell if cell is not None else line 
    return sqlContext.sql(val).explain(detailed_explain)


In [2]:
##############################
###### Load The Delta   ######
##############################
###
### Input delta in folder :  /data 
my_input_delta_table="delta_real_estate_term_definitions"
###
######
##############################Execution##########################
import findspark
findspark.init()
#
import pyspark
from pyspark.sql import functions as pfunc
from pyspark.sql import SQLContext
from pyspark.sql import Window, types
import re
import pandas as pd
import numpy as np
from pandas import DataFrame
from pyspark.sql.types import IntegerType
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from scipy.stats import kstest
from scipy import stats
#
import subprocess
#
sc = pyspark.SparkContext(appName="Business_Dictionary-Delta")
sqlContext = SQLContext(sc)
#
#
internal_delta_files="file:///home/joci/notebooks/data/"+my_input_delta_table
#
# Join with Internal Curation Data in urltopredict staged folder
from pyspark.sql import functions as F
### use version=1
version=1
## .option("versionAsOf", version)
delta_dataframe_df1=sqlContext.read.format("delta").load(internal_delta_files)\
.persist(pyspark.StorageLevel.MEMORY_AND_DISK_2)
#
##.read.format("delta").load("/delta/events")
#
delta_dataframe_df1.printSchema()
delta_dataframe_df1.registerTempTable("real_estate_terms")
#

root
 |-- Real_Estate_Term: string (nullable = true)
 |-- Real_Estate_Definition: string (nullable = true)



In [3]:
%%sql_show
select * from real_estate_terms limit 20

+--------------------+----------------------+
|    Real_Estate_Term|Real_Estate_Definition|
+--------------------+----------------------+
| Acceleration clause|  Also known as an ...|
|   Active contingent|  When a seller acc...|
|Active under cont...|  A house is listed...|
|            Addendum|  If a buyer or sel...|
|Adjustable-rate m...|  The interest rate...|
|     Adjustment date|  This is the date ...|
|        Amortization|  Amortization is t...|
|Annual percentage...|  The annual percen...|
|           Appraisal|  An appraisal on y...|
|        Appreciation|  Appreciation is t...|
|      Assessed value|  An assessment is ...|
|          Assignment|  An assignment is ...|
|  Assumable mortgage|  Assumption is whe...|
|    Balloon mortgage|  Instead of a trad...|
|  Bi-weekly mortgage|  A bi-weekly mortg...|
|         Bridge loan|  A bridge loan is ...|
|              Broker|  A broker has pass...|
|             Buydown|  A buydown is a mo...|
|         Call option|  A call opt

In [5]:
%%sql_show
select * from real_estate_terms limit where Real_Estate_Term like 'Broker%'

+----------------+----------------------+
|Real_Estate_Term|Real_Estate_Definition|
+----------------+----------------------+
|          Broker|  A broker has pass...|
+----------------+----------------------+



In [33]:
####
#### Expose most frequent Real-estate Terms in definitions
####
mywords=sqlContext.sql("select Real_Estate_Definition from real_estate_terms")\
.select(explode(split(col("Real_Estate_Definition"), "\s+")).alias("real_estate_term"))
#.explode("Real_Estate_Definition","term_word") ##( line  => line.split(" ") )
###
mywords.printSchema()
####
filler_words_list=['the','a','of','to','is','or','in','on','for','by','an','The','and','A',\
                   'your','as','that','when','their','it','be','with','you','are','--','It','from','can','usually']
####
wordCountDF = mywords\
.filter(~(col("real_estate_term").isin(filler_words_list)))\
.groupBy("real_estate_term").count().orderBy(col('count').desc())
####        
wordCountDF.show()

root
 |-- real_estate_term: string (nullable = true)

+----------------+-----+
|real_estate_term|count|
+----------------+-----+
|            loan|   37|
|        mortgage|   32|
|        property|   28|
|          seller|   27|
|           buyer|   22|
|        interest|   19|
|         monthly|   19|
|          lender|   18|
|            home|   17|
|        borrower|   17|
|          estate|   15|
|            real|   14|
|            rate|   13|
|       property.|   11|
|           loan.|   11|
|         payment|   11|
|             pay|   10|
|        payments|   10|
|           owner|   10|
|          amount|   10|
+----------------+-----+
only showing top 20 rows



In [13]:
sc.stop()